# Multiple Inheritance

I was reviewing a teammate's code when I stumbled upon a multiple inheritance - he made a class inheriting from 2 classes.
Intuitively I commented that multiple inheritance is bad, but when he (correctly) asked why - I was having a hard time explaining why exactly it is so bad and when can it get complicated.


<figure style="text-align: center;">
    <img src="assets/we_dont_do_that_here.jpg" alt="" width="">
    <figcaption><em>A real footage of me when I saw the multiple inheritance</em>.</figcaption>
</figure>


So we are going to talk about multiple inheritance: how to do it wrong and sometimes correct, and also about the C3 algorithm and mixins.

## MRO and Monotonicity

Consider the following example:

In [1]:
class A: pass


class B(A): pass


class C(A): pass


class D(B, C): pass

# Inheritance graph:
#
#      A
#     / \
#    B   C
#     \ /
#      D
#

This is called a "diamond", because the inheritance graph looks like a diamond shape: `A` is the parent, `B` and `C` are 2 children of it and then `D` is a single child with both `B` and `C` as parents.

This may seem problematic, because we are unsure where `D` get its methods from.
Say `A` implements a `foo` method and `B` and `C` are overriding it, each in its own way; will `D` have the `foo` of `B` or `C`?

In [2]:
class A:
    def foo(self):
        print("A method")


class B(A):
    def foo(self):
        print("B method")


class C(A):
    def foo(self):
        print("C method")


class D(B, C): pass


D().foo()

B method


Well, it's `B` - Python has no problem here.
What's determining which method gets called by `D` is something called the **Method Resolution Order**, or the **MRO** in short.
We can even observe it in the class attributes:

In [3]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

When a `D` method gets called, Python first looks for it in the defined methods of `D`; then if it is not found it looks in inherited classes in the following order: `B`, then `C`, then `A`, then the `object` class that is inherited by everything in Python.

In our case, `B` precedes `C` because this is the order of inheritance in `D`'s definition: `class D(B, C): ...`.

Changing the order in `D` signature would change the MRO:

In [4]:
class D(C, B): pass


D().foo()
D.__mro__

C method


(__main__.D, __main__.C, __main__.B, __main__.A, object)

But there are cases where there is a "disagreement" in the method resolution order, and Python literally won't let us inherit:

In [14]:
# Desired inheritance graph:
#
#      X     Y
#      |     |
#      +--+--+
#         |
#    -----------
#    |         |
#    A         B
#     \       /
#      \     /
#         C

In [15]:
class X: pass


class Y: pass


class A(X, Y): pass


class B(Y, X): pass


class C(A, B): pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases X, Y

What is that *"consistency"* that we fail to create?

When constructing the MRO, we want the following statement to hold:

&nbsp;&nbsp;&nbsp;&nbsp;*If `C1` precedes `C2` in the MRO of some class `C`, then `C1` should also precede `C2` in the MRO of any class inheriting `C`.*

This property is called "monotonicity".
Monotonicity of MROs keeps the behavior of class inheritance simple and intuitive, otherwise it could lead to bugs which are hard to detect.

In our case, the MRO of `B` is `B-Y-X` and the MRO of `A` is `A-X-Y`.
No matter how we order the inheritance for `C`, it violates the monotonicity of one of them: if `X` will precede `Y` then `B` is not monotone, and if `Y` will precede `X` then `A` is not monotone.

Because there is **no possible MRO**, Python raises an error that we fully understand now - cannot create a consistent (monotonic) MRO for `X` and `Y`.

## The C3 Linearization algorithm

The C3 linearization algorithm (which we will call just C3 for short) is explained simply and beautifully in a [document](https://www.python.org/download/releases/2.3/mro/) written by Michele Simionato about how MRO changed between Python 2.2 and 2.3.

Although the explanation by Michele Simionato is very simple, I will also explain it in short here.

First we lay some terminology - if the MRO of some class is `A-B-...-Z` then `A` is called the _head_ of the MRO, and the rest of the MRO `B-...-Z` is called the _tail_.
If a class `A` inherits from multiple classes - for example `A(B,C,D)` - then the MRO of `A` is calculated in the following way:
1. The MRO of `A` starts with `A`.
2. We take the MRO of all inherited classes in their inheritance order: `(MRO(B), MRO(C), MRO(D))`.
3. Construct an MRO from the classes `A` inherit, in order, and append it to the end of the list: `(MRO(B), MRO(C), MRO(D), B-C-D)`.
4. Starting from the first MRO (`MRO(B)`), we check its head:
    * If it does not appear in the tail of any other MRO, then we add it to the MRO of `A`. After that we remove it from *all* MROs.
    * If it does, we move to the head of the next MRO (`MRO(C)`) and repeat step (3).

The process is repeated until all classes were removed and added to the MRO of `A`.
However, there may be cases where it is impossible to find a head which does not appear in any tail - these are the cases where there is no possible MRO and Python raises an exception.

### An Example

Consider the following inheritance (taken from Michele's document):

In [16]:
class F: pass


class E: pass


class D: pass


class C(D, F): pass


class B(D, E): pass


class A(B, C): pass

Our goal is to calculate the MRO of `A`.
We start from the parent classes `D`, `E`, `F` which are trivial:
* `MRO(F) = F`
* `MRO(E) = E`
* `MRO(D) = D`

Then for `C` and `B` we have to apply the C3 algorithm:
* `MRO(C) = C-(MRO(D), MRO(F), D-F) = C-(D, F, D-F)`

According to the algorithm, we start from `D`, see that it does not appear in the tail of any other MRO, and append it to the MRO of `C`.
Now we have:
* `MRO(C) = C-D-(F, F) = C-D-F`

And that's it - this is the MRO of `C`.
Doing the same for `B`:
* `MRO(B) = B-(MRO(D), MRO(E), D-E) = B-(D, E, D-E) = B-D-(E, E) = B-D-E`

Now, the MRO of `A` is where things are getting interesting:
* `MRO(A) = A-(MRO(B), MRO(C), B-C) = A-(B-D-E, C-D-F, B-C)`

* Taking `B` as it's not in any tail:
* `MRO(A) = A-B-(D-E, C-D-F, C)`

We check `D` - it _does_ appear in the tail of the second MRO!
So we move on to the head of the next MRO - `C` - which indeed does not appear in any tail:
* `MRO(A) = A-B-C-(D-E, D-F)`

Now `D` became a valid head, so we can take it:
* `MRO(A) = A-B-C-D-(E, F)`
* `MRO(A) = A-B-C-D-E-F`

And that is the MRO of `A`!
Let's see if our calculation was right:

In [18]:
A.__mro__

(__main__.A,
 __main__.B,
 __main__.C,
 __main__.D,
 __main__.E,
 __main__.F,
 object)

### The Diamond Case and the Disagreement Case